Numerical Methods Project

By **Miguel Polanco**

Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value. 

Rusty Bargain is interested in:

- the quality of the prediction;
- the speed of the prediction;
- the time required for training

## Data preparation

#### importing required libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
import lightgbm as lgb
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')

###  Download  and look at the data

In [3]:
df_car = pd.read_csv('/datasets/car_data.csv')

In [4]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

First information about the dataset after loading in dataframe indicate 16 columns and 354369 rows, 

dtype date and time need setting, missing values in many columns.
nest will be verify the information and fix the duplicates and missing values.

<div class="alert alert-success">
<b>Reviewer's comment</b>

The data was loaded and inspected

</div>

In [5]:
df_car.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
71002,05/04/2016 11:49,750,small,1993,manual,54,clio,70000,2,petrol,renault,no,05/04/2016 00:00,0,36251,05/04/2016 11:49
349134,23/03/2016 16:39,3300,bus,1995,manual,78,transporter,150000,1,gasoline,volkswagen,no,23/03/2016 00:00,0,34225,05/04/2016 19:48
153513,03/04/2016 23:50,2300,small,2003,manual,54,polo,150000,6,petrol,volkswagen,no,03/04/2016 00:00,0,26127,06/04/2016 07:16
219742,21/03/2016 09:37,16999,sedan,2006,auto,286,3er,150000,10,gasoline,bmw,no,21/03/2016 00:00,0,84359,06/04/2016 01:49
198580,27/03/2016 22:43,400,sedan,1997,manual,87,NaN,150000,5,petrol,sonstige_autos,no,27/03/2016 00:00,0,37308,05/04/2016 21:45


In [6]:
df_car = df_car.fillna('unknown')

<div class="alert alert-success">
<b>Reviewer's comment</b>

Missing values were handled reasonably

</div>

In [7]:
df_car.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
295173,16/03/2016 14:54,13500,sedan,1953,manual,34,unknown,90000,10,petrol,sonstige_autos,unknown,16/03/2016 00:00,0,22041,06/04/2016 18:16
251299,03/04/2016 22:31,1600,small,1999,manual,50,lupo,150000,0,petrol,volkswagen,no,03/04/2016 00:00,0,35519,05/04/2016 12:11
107677,11/03/2016 20:55,700,bus,2003,manual,101,other,150000,10,petrol,mazda,yes,11/03/2016 00:00,0,99994,06/04/2016 05:46
267565,05/03/2016 20:49,13999,suv,1992,auto,136,g_klasse,125000,12,gasoline,mercedes_benz,no,05/03/2016 00:00,0,35745,07/03/2016 12:33
350670,30/03/2016 10:44,2800,wagon,1995,auto,290,other,150000,10,petrol,audi,no,30/03/2016 00:00,0,83374,07/04/2016 01:15


In [8]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        354369 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            354369 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              354369 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           354369 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        354369 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

The date alone does not provide us with substantial information, but the duration between the publication of a listing and its most recent activity can indicate the speed at which cars are being sold. This time gap can be influenced by various factors, including price policy. For instance, if the listings are priced too high, they are unlikely to be sold quickly.

<div class="alert alert-success">
<b>Reviewer's comment</b>

Good point!

</div>

In [9]:
# duplicates?
df_car.duplicated().sum()

262

In [10]:
df_car.drop_duplicates(inplace=True)

df_car.shape

(354107, 16)

**Data preparation**

Set date time field as dtype datetime.

add a column 'DeltaDays' 

In [11]:
df_car['DateCreated'] = pd.to_datetime(df_car['DateCreated'], format='%d/%m/%Y %H:%M')
df_car['LastSeen'] = pd.to_datetime(df_car['LastSeen'], format='%d/%m/%Y %H:%M')
df_car['DeltaDays'] = (df_car['LastSeen']-df_car['DateCreated']).dt.days

In [12]:
# check no negative values
np.any(df_car['DeltaDays'] < 0)

False

In [13]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354107 entries, 0 to 354368
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        354107 non-null  object        
 1   Price              354107 non-null  int64         
 2   VehicleType        354107 non-null  object        
 3   RegistrationYear   354107 non-null  int64         
 4   Gearbox            354107 non-null  object        
 5   Power              354107 non-null  int64         
 6   Model              354107 non-null  object        
 7   Mileage            354107 non-null  int64         
 8   RegistrationMonth  354107 non-null  int64         
 9   FuelType           354107 non-null  object        
 10  Brand              354107 non-null  object        
 11  NotRepaired        354107 non-null  object        
 12  DateCreated        354107 non-null  datetime64[ns]
 13  NumberOfPictures   354107 non-null  int64   


In our scenario, the focus is on the listing price, while the dates themselves will not be considered as part of the features.

In [14]:
target = df_car['Price']
features = df_car.drop(columns  = ['Price', 'DateCreated', 'LastSeen','DateCrawled'], axis = 1) 

<div class="alert alert-success">
<b>Reviewer's comment</b>

Alright!

</div>

The dataset can be divided into training, testing, and validation sets in a ratio of 3:1:1.

In [15]:
features_train, features_rest, target_train, target_rest = train_test_split(features, target, test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(features_rest,target_rest, test_size=0.5, random_state=12345)

<div class="alert alert-success">
<b>Reviewer's comment</b>

The data was split into train, validation and test set

</div>

Numeric features need to be scaled for normalization purposes.

In [16]:
features_train.head()

,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode,DeltaDays
62746,small,2007,auto,71,fortwo,125000,12,petrol,smart,no,0,71364,19
324587,small,2005,manual,60,2_reihe,70000,1,petrol,peugeot,no,0,13581,1
89315,small,2001,manual,60,1_reihe,150000,6,petrol,peugeot,no,0,80686,3
217912,unknown,2018,unknown,58,corsa,80000,1,petrol,opel,yes,0,59757,3
202884,sedan,2001,manual,140,omega,150000,6,petrol,opel,no,0,49084,4


In [17]:
numeric_cols = ['RegistrationYear','Power','Mileage','RegistrationMonth','PostalCode','DeltaDays' ]

scaler = StandardScaler()
scaler.fit(features_train[numeric_cols])


StandardScaler()

In [18]:
features_train[numeric_cols] = scaler.transform(features_train[numeric_cols])
features_test[numeric_cols] = scaler.transform(features_test[numeric_cols])
features_valid[numeric_cols] = scaler.transform(features_valid[numeric_cols])

<div class="alert alert-success">
<b>Reviewer's comment</b>

Scaling was applied correctly

</div>

In [19]:
features_train.head()

,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode,DeltaDays
62746,small,0.030174,auto,-0.208856,fortwo,-0.085235,1.685739,petrol,smart,no,0,0.809488,1.186818
324587,small,0.008102,manual,-0.267675,2_reihe,-1.538183,-1.264328,petrol,peugeot,no,0,-1.432339,-0.880204
89315,small,-0.036042,manual,-0.267675,1_reihe,0.575197,0.076611,petrol,peugeot,no,0,1.171156,-0.650535
217912,unknown,0.151569,unknown,-0.278369,corsa,-1.274011,-1.264328,petrol,opel,yes,0,0.359167,-0.650535
202884,sedan,-0.036042,manual,0.160097,omega,0.575197,0.076611,petrol,opel,no,0,-0.054917,-0.535700


In [20]:
categ_fiatures = list(features.loc[:, features.dtypes == 'object'].columns.values)

In [21]:
categ_fiatures

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

Categorical feature encoding will not be performed at this stage as various algorithms necessitate distinct encoding methods or have their own built-in implementations.

<div class="alert alert-success">
<b>Reviewer's comment</b>

Ok!

</div>

## Model training

**LightGBM**

Given that this algorithm possesses its own implementation of feature encoding, it is advisable to alter the data type of certain columns to 'category'.

In [22]:
for column in categ_fiatures:
    features_train[column] = pd.Series(features_train[column], dtype="category")
    features_test[column] = pd.Series(features_test[column], dtype="category")
    features_valid[column] = pd.Series(features_valid[column], dtype="category")

In [23]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212464 entries, 62746 to 217680
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   VehicleType        212464 non-null  category
 1   RegistrationYear   212464 non-null  float64 
 2   Gearbox            212464 non-null  category
 3   Power              212464 non-null  float64 
 4   Model              212464 non-null  category
 5   Mileage            212464 non-null  float64 
 6   RegistrationMonth  212464 non-null  float64 
 7   FuelType           212464 non-null  category
 8   Brand              212464 non-null  category
 9   NotRepaired        212464 non-null  category
 10  NumberOfPictures   212464 non-null  int64   
 11  PostalCode         212464 non-null  float64 
 12  DeltaDays          212464 non-null  float64 
dtypes: category(6), float64(6), int64(1)
memory usage: 14.4 MB


Let us proceed with training a model using two distinct sets of parameters and observe the duration it takes.

In [24]:
# defining parameters 
params1 = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 1000,
    'learning_rate': 0.05,
    'metric': 'root_mean_squared_error',
}
lgb_train = lgb.Dataset(features_train, target_train,  categorical_feature=categ_fiatures)
lgb_eval = lgb.Dataset(features_valid, target_valid, reference=lgb_train,  categorical_feature=categ_fiatures)

In [25]:
%%time
# fitting the model
lgb1 = lgb.train(params1,
                 train_set=lgb_train,
                 valid_sets=lgb_eval)
  

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009087 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1019
[LightGBM] [Info] Number of data points in the train set: 212464, number of used features: 12
[LightGBM] [Info] Start training from score 4419.930699
[1]	valid_0's rmse: 4311
[2]	valid_0's rmse: 4140.32
[3]	valid_0's rmse: 3979.53
[4]	valid_0's rmse: 3828.15
[5]	valid_0's rmse: 3685.78
[6]	valid_0's rmse: 3551.89
[7]	valid_0's rmse: 3425.96
[8]	valid_0's rmse: 3307.88
[9]	valid_0's rmse: 3196.66
[10]	valid_0's rmse: 3092.58
[11]	valid_0's rmse: 2994.95
[12]	valid_0's rmse: 2903.6
[13]	valid_0's rmse: 2818.57
[14]	valid_0's rmse: 2738.59
[15]	valid_0's rmse: 2664.09
[16]	valid_0's rmse: 2594.51
[17]	valid_0's rmse: 2528.18
[18]	valid_0's rmse: 2467.27
[19]	valid_0's rmse: 2409.72
[20]	valid_0's rmse: 2356.51
[21]	valid_0's rmse:

In [26]:
# defining parameters 
params2 = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 500,
    'learning_rate': 0.1,
    'metric': 'root_mean_squared_error'
}
lgb_train = lgb.Dataset(features_train, target_train,  categorical_feature=categ_fiatures)
lgb_eval = lgb.Dataset(features_valid, target_valid, reference=lgb_train,  categorical_feature=categ_fiatures)



In [27]:
%%time
lgb2 = lgb.train(params2,
                train_set=lgb_train, valid_sets=lgb_eval)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008261 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1019
[LightGBM] [Info] Number of data points in the train set: 212464, number of used features: 12
[LightGBM] [Info] Start training from score 4419.930699
[1]	valid_0's rmse: 4136.41
[2]	valid_0's rmse: 3822.61
[3]	valid_0's rmse: 3545.76
[4]	valid_0's rmse: 3301.79
[5]	valid_0's rmse: 3087.85
[6]	valid_0's rmse: 2900.87
[7]	valid_0's rmse: 2734.81
[8]	valid_0's rmse: 2590.39
[9]	valid_0's rmse: 2465.04
[10]	valid_0's rmse: 2356.1
[11]	valid_0's rmse: 2263.01
[12]	valid_0's rmse: 2181.53
[13]	valid_0's rmse: 2112.67
[14]	valid_0's rmse: 2051.08
[15]	valid_0's rmse: 1999.38
[16]	valid_0's rmse: 1955.78
[17]	valid_0's rmse: 1917.34
[18]	valid_0's rmse: 1884.37
[19]	valid_0's rmse: 1856.47
[20]	valid_0's rmse: 1831.52
[21]	valid_0's rm

**CatBoost**

This algorithm possesses its unique implementation of encoding categorical features.

In [28]:
%%time
catboost = CatBoostRegressor(loss_function="RMSE", iterations=300)
catboost.fit(features_train, target_train, cat_features=categ_fiatures)

Learning rate set to 0.254092
0:	learn: 3829.5157764	total: 618ms	remaining: 3m 4s
1:	learn: 3351.9088222	total: 1.24s	remaining: 3m 4s
2:	learn: 3000.6308974	total: 1.74s	remaining: 2m 52s
3:	learn: 2741.1843109	total: 2.15s	remaining: 2m 38s
4:	learn: 2568.7118042	total: 2.62s	remaining: 2m 34s
5:	learn: 2436.7246243	total: 3.16s	remaining: 2m 34s
6:	learn: 2341.4080738	total: 3.66s	remaining: 2m 33s
7:	learn: 2260.9153642	total: 4.17s	remaining: 2m 32s
8:	learn: 2203.7549591	total: 4.59s	remaining: 2m 28s
9:	learn: 2158.8011820	total: 5.01s	remaining: 2m 25s
10:	learn: 2130.1556491	total: 5.46s	remaining: 2m 23s
11:	learn: 2104.2713801	total: 5.93s	remaining: 2m 22s
12:	learn: 2085.7228730	total: 6.36s	remaining: 2m 20s
13:	learn: 2065.8947737	total: 6.71s	remaining: 2m 17s
14:	learn: 2050.8575997	total: 7.17s	remaining: 2m 16s
15:	learn: 2036.2852110	total: 7.66s	remaining: 2m 15s
16:	learn: 2026.9754076	total: 8.09s	remaining: 2m 14s
17:	learn: 2018.3891240	total: 8.5s	remaining: 

**Random Forest**

This particular model necessitates the utilization of one-hot encoding (OHE) for categorical features. Let us employ this technique and proceed with training the model using a specific pair of parameter values. Although GridSearchCV can be employed to identify the optimal set of hyperparameters, it should be noted that its implementation can be time-consuming, often taking several hours.

In [29]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(features_train)

features_train_ordinal = encoder.transform(features_train)
features_test_ordinal = encoder.transform(features_test)
features_valid_ordinal = encoder.transform(features_valid)

In [30]:
%%time

rf1 = RandomForestRegressor(max_depth=10, random_state=12345, n_estimators = 100)
rf1.fit(features_train_ordinal, target_train)

CPU times: user 6min 2s, sys: 423 ms, total: 6min 2s
Wall time: 6min 3s


RandomForestRegressor(max_depth=10, random_state=12345)

In [31]:
%%time

rf2 = RandomForestRegressor(max_depth=5, random_state=12345, n_estimators = 50)
rf2.fit(features_train_ordinal, target_train)

CPU times: user 36.7 s, sys: 75.7 ms, total: 36.8 s
Wall time: 36.8 s


RandomForestRegressor(max_depth=5, n_estimators=50, random_state=12345)

<div class="alert alert-success">
<b>Reviewer's comment</b>

Great, you tried a couple of different models, applied appropriate categorical feature encoding and did some hyperparameter tuning using the validation set

</div>

## Model analysis

**Linear Regression as sanity check**

This algorithm is not the optimal choice for resolving our issue. Therefore, if gradient boosting techniques exhibit inferior performance, it indicates that an error has occurred.

In [32]:
model = LinearRegression()
model.fit(features_train_ordinal, target_train)
pred = model.predict(features_valid_ordinal)
rmse = (mean_squared_error(target_valid,pred))**0.5
rmse

2139.7087285981534

**LightGBM**

compute the RMSE for 2 sets of parameters.

In [33]:
target_pred = lgb1.predict(features_valid)

# accuracy check
mse = mean_squared_error(target_valid, target_pred)
rmse = mse**(0.5)

print("RMSE: %.2f" % rmse) 

RMSE: 1652.48


In [34]:
target_pred = lgb2.predict(features_valid)

# accuracy check
mse = mean_squared_error(target_valid, target_pred)
rmse = mse**(0.5)

print("RMSE: %.2f" % rmse) 
 

RMSE: 1649.50


The second set of parameters in LightGBM not only improves the training time but also yields a slightly better RMSE score.

**CatBoost**

In [35]:
target_predict = catboost.predict(features_valid)

In [36]:
# accuracy check

mse = mean_squared_error(target_valid, target_predict)
rmse = mse**0.5

print("RMSE: %.2f" % rmse)

RMSE: 1734.55


Despite undergoing a more extensive training duration compared to LightGBM, it exhibits inferior performance with a higher root mean square error (RMSE).

**RandomForest**

In [37]:
target_predict = rf1.predict(features_valid_ordinal)

In [38]:
# accuracy check
mse = mean_squared_error(target_valid, target_predict)
rmse = mse**.5

print("RMSE: %0.2f" % rmse)

RMSE: 2840.62


In [39]:
target_predict = rf2.predict(features_valid_ordinal)

# accuracy check
mse = mean_squared_error(target_valid, target_predict)
rmse = mse**(0.5)

print("RMSE: %.2f" % rmse)

RMSE: 3426.50


**Conclusion**

An alternative approach to gradient boosting can enhance the efficiency of model training and yield superior RMSE scores compared to Linear Regression. Random Forest is not a viable option due to the extensive time required to identify optimal hyperparameters and subsequently train the model. In terms of both training speed and quality, LightGBM demonstrated the most impressive performance.


<table>
  <tr>
   <td>Model
   </td>
   <td>RMSE - the quality of the prediction
   </td>
   <td>the time required for training
   </td>
  </tr>
  <tr>
   <td>LightGBM parameter 1
   </td>
   <td>1652.48
   </td>
   <td>41.8 sec
   </td>
  </tr>
  <tr>
   <td>LightGBM parameter 2
   </td>
   <td>1649.50
   </td>
   <td>29 sec
   </td>
  </tr>
  <tr>
   <td>CatBoost
   </td>
   <td>1734.55
   </td>
   <td>2min 12s
   </td>
  </tr>
  <tr>
   <td>RandomForest Parameter 1
   </td>
   <td>2840.62
   </td>
   <td>6min 2s
   </td>
  </tr>
  <tr>
   <td>RandomForest Parameter 2
   </td>
   <td>3426.50
   </td>
   <td>36.8 s
   </td>
  </tr>
</table>

# Checklist

Type 'x' to check. Then press Shift+Enter.

- [x]  Jupyter Notebook is open
- [ ]  Code is error free
- [ ]  The cells with the code have been arranged in order of execution
- [ ]  The data has been downloaded and prepared
- [ ]  The models have been trained
- [ ]  The analysis of speed and quality of the models has been performed